In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter(action="ignore")
import category_encoders as ce
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 170)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df_ = pd.read_csv("car_evaluation.csv")

In [3]:
df=df_
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [4]:
# Rename column names
df = df.rename(columns={"vhigh": "buying", "vhigh.1": "maint","2":"doors","2.1":"persons","small":"lug_boot","low":"safety","unacc":"class_values"})
df.head()

,buying,maint,doors,persons,lug_boot,safety,class_values
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   buying        1727 non-null   object
 1   maint         1727 non-null   object
 2   doors         1727 non-null   object
 3   persons       1727 non-null   object
 4   lug_boot      1727 non-null   object
 5   safety        1727 non-null   object
 6   class_values  1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


All are object

In [6]:
df.shape

(1727, 7)

In [7]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

for col in df.columns:
    cat_summary(df, col)

       buying  Ratio
high      432 25.014
med       432 25.014
low       432 25.014
vhigh     431 24.957
##########################################
       maint  Ratio
high     432 25.014
med      432 25.014
low      432 25.014
vhigh    431 24.957
##########################################
       doors  Ratio
3        432 25.014
4        432 25.014
5more    432 25.014
2        431 24.957
##########################################
      persons  Ratio
4         576 33.353
more      576 33.353
2         575 33.295
##########################################
       lug_boot  Ratio
med         576 33.353
big         576 33.353
small       575 33.295
##########################################
      safety  Ratio
med      576 33.353
high     576 33.353
low      575 33.295
##########################################
       class_values  Ratio
unacc          1209 70.006
acc             384 22.235
good             69  3.995
vgood            65  3.764
##########################################


In [8]:
df.isnull().sum()

buying          0
maint           0
doors           0
persons         0
lug_boot        0
safety          0
class_values    0
dtype: int64

Eksik veri yok.

In [45]:
X=df.drop(["class_values"],axis=1)
y=df["class_values"]

train test split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, validation_curve
warnings.simplefilter(action='ignore', category=Warning)

In [47]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# Random Forests

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=17).fit(X_train,y_train)

In [26]:
y_pred = rf_model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
print("Accuracy Score: ",accuracy_score(y_test, y_pred))

Accuracy Score:  0.8959537572254336


Hiperparametre Optimizasyonu

In [28]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 17,
 'verbose': 0,
 'warm_start': False}

In [31]:
rf_params = {"max_depth":[None,5,8,10],
            "max_features":["auto",3,5,7],
            "n_estimators":[100,200,300],
            "min_samples_split":[2,5,7,10,15,20]}

In [32]:
rf_best_grid = GridSearchCV(rf_model,rf_params,cv=5,n_jobs=-1,verbose=False).fit(X_train,y_train)

In [33]:
rf_best_grid.best_params_

{'max_depth': None,
 'max_features': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

In [34]:
rf_final = rf_model.set_params(**rf_best_grid.best_params_,random_state=17).fit(X_train,y_train)

In [36]:
y_pred = rf_final.predict(X_test)

In [37]:
print("Accuracy Score: ",accuracy_score(y_test, y_pred))

Accuracy Score:  0.9450867052023122


### XgBoost

In [49]:
from xgboost import XGBClassifier
from sklearn import preprocessing
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)
xgb_model = XGBClassifier(random_state=17).fit(X_train,y_train)

In [50]:
y_pred = xgb_model.predict(X_test)

In [51]:
print("Accuracy Score: ",accuracy_score(y_test, y_pred))

Accuracy Score:  0.9739884393063584


Hiperparametre Optimizasyonu

In [52]:
xgb_model.get_params()

{'objective': 'multi:softprob',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': 17,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [56]:
xgb_params = {"learning_rate":[None,0.1,0.01,0.001],
              "max_depth":[None,5,10,15,20],
            "colsample_bytree":[None,0.7,1,2],
            "n_estimators":[100,200,300]}

In [57]:
xgb_best_grid = GridSearchCV(xgb_model,xgb_params,cv=5,n_jobs=-1,verbose=False).fit(X_train,y_train)

In [58]:
xgb_best_grid.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': None,
 'max_depth': None,
 'n_estimators': 200}

In [59]:
xgb_final = xgb_model.set_params(**xgb_best_grid.best_params_,random_state=17).fit(X_train,y_train)

In [60]:
y_pred = xgb_final.predict(X_test)

In [61]:
print("Accuracy Score: ",accuracy_score(y_test, y_pred))

Accuracy Score:  0.9710982658959537


In [62]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95        76
           1       0.79      0.95      0.86        20
           2       1.00      0.99      1.00       236
           3       1.00      0.71      0.83        14

    accuracy                           0.97       346
   macro avg       0.93      0.90      0.91       346
weighted avg       0.97      0.97      0.97       346

